# 第10回講義 宿題

## 課題. 変分オートエンコーダ（VAE）でFasionMNISTの画像を生成せよ

## Homework

変分オートエンコーダ（VAE）により, FashionMNISTの画像を生成してみましょう.。

## ルール

- 訓練データはx_train、テストデータはx_testで与えられます。
- 下のセルで指定されているx_train以外の学習データは使わないでください。

## 目標値

NLL（負の対数尤度） 235

## 提出方法

- 2つのファイルを提出していただきます。
    1. テストデータ（x_test）について、VAEで生成した画像をcsvファイル（ファイル名：submission.csv）として保存し（1画像が1行に対応）、**Homeworkタブからchap10を選択して**提出してください。
    2. それに対応するpythonのコードをsubmission_code.pyとして保存し、**Homeworkタブからchap10 (code)を選択して**提出してください。 
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。


## 評価について

- 評価は生成画像のテストデータに対するNLL（負の対数尤度）で行います.  $-\sum_{i=1}^D x_i\log \hat{x}_i + (1-x_i)\log (1-\hat{x}_i)$
- 毎日夜24時にテストデータの一部に対するNLLでLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対するNLLでLeader Boardを更新します。これを最終的な評価とします。

## サンプルコード

- 次のセルで指定されているx_trainのみを使って学習させてください.
- submission.csvの出力場所は適宜変更してください.

In [5]:
# このデータの読み込み部分は修正しないでください
import numpy as np
import pandas as pd

def load_mnist():

    # 学習データ
    x_train = np.load('/root/userspace/public/chap10/data/x_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap10/data/x_test.npy')

    x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
    x_test = (x_test.reshape(-1, 784) / 255).astype(np.float32)

    return (x_train, x_test)

x_train, x_test = load_mnist()

## VAEの実装

In [6]:
#%%writefile /root/userspace/chap10/submission_code.py

import tensorflow as tf
import csv

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#乱数について追加
random_seed = 10 #1~10で10が一番
np.random.seed(random_seed)
tf.set_random_seed(random_seed) 

# rng = np.random.RandomState(1234)

### define layers ###
tf.reset_default_graph()
z_dim = 10 #5とかよかったけど。defalutは10

def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

def encoder(x):# WRITE ME
    with tf.variable_scope('Encoder', reuse=tf.AUTO_REUSE):
        h1 = tf.layers.Dense(units=200, activation=tf.nn.relu)(x)
        h2 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h1)
        mean = tf.layers.Dense(units=z_dim)(h2)
        var = tf.layers.Dense(units=z_dim, activation=tf.nn.softplus)(h2)
    return mean, var    

def sampling_z(mean, var):# WRITE ME
    epsilon = tf.random_normal(shape=tf.shape(mean))
    z = mean + tf.sqrt(var) * epsilon
    return z    

def decoder(z):# WRITE ME
    with tf.variable_scope('Decoder', reuse=tf.AUTO_REUSE):
        h3 = tf.layers.Dense(units=200, activation=tf.nn.relu)(z)
        h4 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h3)
        y = tf.layers.Dense(units=784, activation=tf.nn.sigmoid)(h4)
    return y    

def lower_bound(x):
    #Encode
    mean, var = encoder(x)
    KL = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf_log(var) - mean**2 - var, axis=1))
    #Z
    z = sampling_z(mean, var)
    #Decode
    y = decoder(z)
    reconstruction = tf.reduce_mean(tf.reduce_sum(x * tf_log(y) + (1 - x) * tf_log(1 - y), axis=1))
    
    lower_bound = [-KL, reconstruction]
    return lower_bound

# zcaとかBatchNormを導入しようとしたが、やりかたわからず
    
### training ###
#学習データと検証データに分割
x_train, x_valid = train_test_split(x_train, test_size=0.1)

x = tf.placeholder(tf.float32, [None, 784])
lower_bound = lower_bound(x)

cost = -tf.reduce_sum(lower_bound)

# optimizer = tf.train.AdamOptimizer()
# train = optimizer.minimize(cost)
optimizer = tf.train.AdamOptimizer()
grads = optimizer.compute_gradients(cost)
clipped_grads = [(tf.clip_by_value(grad_val, -1., 1.), var) for grad_val, var in grads]
train = optimizer.apply_gradients(clipped_grads)

valid = tf.reduce_sum(lower_bound)

batch_size =100

n_batches = x_train.shape[0] // batch_size
n_epochs = 20

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for epoch in range(n_epochs):# WRITE ME
    rng.shuffle(x_train)
    lower_bound_all = []
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        _, lowerbound = sess.run([train, lower_bound], feed_dict={x: x_train[start:end]})
        lower_bound_all.append(lowerbound)
    lower_bound_all = np.mean(lower_bound_all, axis=0)
    lower_bound_valid = sess.run(valid, feed_dict={x: x_valid[0:100]})
    print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
          (epoch+1, np.sum(lower_bound_all), lower_bound_all[0], lower_bound_all[1], lower_bound_valid))    
    
### sampling ###
x = tf.placeholder(tf.float32, [None, 784])
sample_z_func = encoder(x)

z = tf.placeholder(tf.float32, [None, z_dim])
sample_x_func = decoder(z)

# Encode
mean, var = sess.run(sample_z_func, feed_dict={x: x_test})
sample_z = mean

# Decode
sample_x = sess.run(sample_x_func, feed_dict={z: sample_z})


### to_csv ###
with open('/root/userspace/chap10/materials/submission.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(sample_x.reshape(-1, 28*28).tolist())

EPOCH:1, Train Lower Bound:-282.338257, (-10.305874, -272.032379), Valid Lower Bound:-244.587112
EPOCH:2, Train Lower Bound:-253.468246, (-11.575931, -241.892319), Valid Lower Bound:-237.206589
EPOCH:3, Train Lower Bound:-249.125122, (-11.994110, -237.131012), Valid Lower Bound:-234.741486
EPOCH:4, Train Lower Bound:-246.956818, (-12.170147, -234.786667), Valid Lower Bound:-232.759689
EPOCH:5, Train Lower Bound:-245.535858, (-12.270274, -233.265579), Valid Lower Bound:-232.071609
EPOCH:6, Train Lower Bound:-244.508667, (-12.377619, -232.131042), Valid Lower Bound:-230.356216
EPOCH:7, Train Lower Bound:-243.702499, (-12.447305, -231.255188), Valid Lower Bound:-229.427643
EPOCH:8, Train Lower Bound:-243.132156, (-12.516734, -230.615417), Valid Lower Bound:-229.609695
EPOCH:9, Train Lower Bound:-242.582382, (-12.531636, -230.050751), Valid Lower Bound:-228.048294
EPOCH:10, Train Lower Bound:-242.185364, (-12.581368, -229.603989), Valid Lower Bound:-227.918427
EPOCH:11, Train Lower Bound:-